In [15]:
import re
import os
import yaml
import requests
import pandas as pd

from io import StringIO
from yaml import Loader

In [91]:
params = yaml.load(open("../params.yaml", 'r'), Loader=Loader)

In [4]:
with requests.Session() as session: 
    for link in params['data_links']:
        print(link)
        download = session.get(link)
        
        df = pd.read_csv(
            StringIO(download.content.decode(encoding='utf-8', errors='replace')),
            encoding= 'unicode_escape',
            low_memory=False
        )

        if 'vehicle' in link:
            file_type = 'vehicles'
        elif 'casualty' in link:
            file_type = 'casualties'
        elif 'attendant' in link:
            file_type = 'collisions'
            
        display(df.head())
        print(df.shape)

        year = re.findall("20\d{2}", link)[0]
        
        df.to_csv(f'../data/raw-data/{file_type}-{year}.csv', index=False)

https://content.tfl.gov.uk/jan-dec-2021-gla-data-extract-vehicle.csv


,GLA Area 2021,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,_Collision Id,Borough Name,Borough Number,Easting,Northing,_Vehicle Id,Vehicle Type Banded,Vehicle Type,Vehicle Manoeuvres,Vehicle Skidding and Overturning,Vehicle Restricted Lane,Location of Vehicle at First Impact,Vehicle Hit Object in Carriageway,Vehicle Leaving Carriageway,Vehicle Hit Object Off Carriageway,First Point of Impact,Journey Purpose,Driver Gender,Driver Age,Driver age (Banded)
1,1210287148,Barnet,30,521508,193079,1,Car,Car,Going Ahead Right Bend,Skid/Over,On Main CW - Not Rest.,Jct App,Hit Parked Veh,Did Not Leave,None,Front Hit First,Other/Not Known,Male,19,17-24
2,1210287148,Barnet,30,521508,193079,2,Car,Car,Parked,No Skidding/Overturn,On Main CW - Not Rest.,Jct App,None,Did Not Leave,None,Front Hit First,Unknown,Not Traced,NaN,Unknown
3,1210287148,Barnet,30,521508,193079,3,Car,Car,Parked,No Skidding/Overturn,On Main CW - Not Rest.,Jct App,None,Did Not Leave,None,Back Hit First,Unknown,Not Traced,NaN,Unknown
4,1210287149,Tower Hamlets,5,535379,180783,1,Car,Car,Going Ahead Other,No Skidding/Overturn,On Main CW - Not Rest.,Jct Mid,None,Did Not Leave,None,Front Hit First,Other/Not Known,Male,32,25-34


(42833, 20)
https://content.tfl.gov.uk/jan-dec-2021-gla-data-extract-casualty.csv


,GLA area 2021,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,_Collision Id,Borough Name,Borough Number,Easting,Northing,_Casualty Id,_Casualty Class,Casualty Gender,Casualty Age (5 Year Bands),Casualty Age,_Casualty Count,_Casualty Severity,Pedestrian Location,Pedestrian Movement,Casualty Mode of Travel
1,1210287148,Barnet,30,521508,193079,1,Driver/Rider,Male,15-19,19,1,Slight,Not Applicable,Not Applicable,Car
2,1210287149,Tower Hamlets,5,535379,180783,1,Passenger,Male,25-29,25,1,Slight,Not Applicable,Not Applicable,Car
3,1210287149,Tower Hamlets,5,535379,180783,2,Driver/Rider,Male,20-24,21,1,Slight,Not Applicable,Not Applicable,Car
4,1210287149,Tower Hamlets,5,535379,180783,3,Passenger,Male,15-19,17,1,Serious,Not Applicable,Not Applicable,Car


(26673, 15)
https://content.tfl.gov.uk/jan-dec-2021-gla-data-extract-attendant.csv


...

(43445, 21)
https://content.tfl.gov.uk/2011-gla-data-extract-casualty.csv


,AREFNO,Borough,Boro,Easting,Northing,CREFNO,Casualty Class,Casualty Sex,Casualty Age (Banded),Casualty Age,No. of Casualties,Casualty Severity,Ped. Location,Ped. Movement,Mode of Travel,
0,0111CP00001,CITY OF LONDON,0,532740,181040,1,1 Driver/Rider,2 Female,25-59,32,1,2 Serious,-2 Unknown,-2 N/A,3 Powered 2 Wheeler,
1,0111CP00001,CITY OF LONDON,0,532740,181040,2,3 Pedestrian,1 Male,25-59,38,1,3 Slight,01 Crossing Road On Ped Xing,4 From Drivers O/Side Msk,1 Pedestrian,
2,0111CP00002,CITY OF LONDON,0,532100,181890,1,3 Pedestrian,1 Male,25-59,28,1,3 Slight,05 Crossing Road (Not On Xing),2 From Drivers N/Side Msk,1 Pedestrian,
3,0111CP00003,CITY OF LONDON,0,533560,181150,1,3 Pedestrian,2 Female,25-59,27,1,3 Slight,04 Crossing Road Within 50m Xing,1 From Drivers N/Side,1 Pedestrian,
4,0111CP00004,CITY OF LONDON,0,533280,181600,1,1 Driver/Rider,1 Male,25-59,54,1,3 Slight,-2 Unknown,-2 N/A,5 Taxi,


(29257, 16)
https://content.tfl.gov.uk/2011-gla-data-extract-attendant.csv


,AREFNO,Borough,Boro,Easting,Northing,Location,Accident Severity,No. of Casualties in Acc.,No. of Vehicles in Acc.,Accident Date,...,Junction Detail,Junction Control,Road Class 2,Road No. 2,Ped. Crossing Decoded,Light Conditions (Banded),Weather,Road Surface,Special Conditions,C/W Hazard
0,0111CP00001,CITY OF LONDON,0,532740,181040,KING WILLIAM STREET J/W LOMBARD STREET.,2 Serious,2,2,07-Jan-11,...,6 Crossroads,2 Auto Sig,C,0,5 Pedn Phase At Ats,1 Daylight,2 Raining,2 Road-Wet,0 None,0 None
1,0111CP00002,CITY OF LONDON,0,532100,181890,ALDERSGATE STREET 50 METRES NORTH J/W BEECH ST...,3 Slight,1,1,06-Jan-11,...,0 No Jun In 20m,-2 Not Applicable,-2 Not Applicable,0,0 No Xing Facility In 50m,1 Daylight,2 Raining,2 Road-Wet,0 None,0 None
2,0111CP00003,CITY OF LONDON,0,533560,181150,ALDGATE J/W JEWRY STREET.,3 Slight,1,1,10-Jan-11,...,3 T/Stag Jun,4 Give Way/Uncontrolled,C,0,4 Pelican Or Similar,1 Daylight,1 Fine,1 Road-Dry,0 None,0 None
3,0111CP00004,CITY OF LONDON,0,533280,181600,BISHOPSGATE J/W NEW STREET.,3 Slight,1,2,09-Jan-11,...,3 T/Stag Jun,2 Auto Sig,Unclassified,0,5 Pedn Phase At Ats,2 Dark,1 Fine,1 Road-Dry,0 None,0 None
4,0111CP00005,CITY OF LONDON,0,532660,181330,MOORGATE J/W KING'S ARM YARD.,3 Slight,1,1,10-Jan-11,...,6 Crossroads,4 Give Way/Uncontrolled,Unclassified,0,0 No Xing Facility In 50m,1 Daylight,1 Fine,1 Road-Dry,0 None,0 None


(24443, 27)


In [5]:
# test random
pd.read_csv('../data/raw-data/collisions-2014.csv').head()

,AREFNO,Borough,Boro,Easting,Northing,Location,Accident Severity,No. of Casualties in Acc.,No. of Vehicles in Acc.,Accident Date,...,Junction Detail,Junction Control,Road Class 2,Road No. 2,Ped. Crossing Decoded,Light Conditions (Banded),Weather,Road Surface,Special Conditions,C/W Hazard
0,0114CP00001,CITY OF LONDON,0,533540,181230,ST BOTOLPH STREET J/W HOUNDSDITCH,3 Slight,1,1,01-Jan-14,...,3 T/Stag Jun,4 Give Way/Uncontrolled,A,1211,0 No Xing Facility In 50m,2 Dark,1 Fine,2 Road-Wet,0 None,0 None
1,0114CP00002,CITY OF LONDON,0,532680,181430,MOORGATE J/W GREAT SWAN ALLEY,3 Slight,1,1,08-Jan-14,...,3 T/Stag Jun,4 Give Way/Uncontrolled,C,0,0 No Xing Facility In 50m,1 Daylight,1 Fine,1 Road-Dry,0 None,0 None
2,0114CP00003,CITY OF LONDON,0,532090,181830,ALDERGATE STREET J/W LONG LANE,2 Serious,1,1,09-Jan-14,...,3 T/Stag Jun,2 Auto Sig,C,0,5 Pedn Phase At Ats,2 Dark,1 Fine,1 Road-Dry,0 None,0 None
3,0114CP00004,CITY OF LONDON,0,531770,180950,QUEEN VICTORIA STREET J/W PUDDLE DOCK,2 Serious,1,2,08-Jan-14,...,3 T/Stag Jun,2 Auto Sig,C,0,5 Pedn Phase At Ats,1 Daylight,1 Fine,1 Road-Dry,0 None,0 None
4,0114CP00005,CITY OF LONDON,0,533130,180920,FENCHURCH STREET J/W ROOD LANE,3 Slight,1,1,15-Jan-14,...,3 T/Stag Jun,4 Give Way/Uncontrolled,C,0,0 No Xing Facility In 50m,1 Daylight,1 Fine,1 Road-Dry,0 None,0 None


In [163]:
def extract_columns(df, schema):
    inverse_schema = {}
    for key, val in collision_schema.items():
        for v in val:
            inverse_schema[v] = key
        
    df = df.rename(columns=inverse_schema)
    columns = schema.keys()
    return df[columns]


def format_column_names(df):
    cols = []
    for col in df.columns:
        col = col.strip() # strip trailing spaces
        col = col.lower()
        col = col.lstrip('_')  # string underscores infront of col name
        col = re.sub(r'[^\w\s]', '', col)  # remove punctuation
        col = col.replace(' ', '_')  # replace spaces between words w/ underscores
        cols.append(col)
    df.columns = cols
    return df


def format_severity(severity):
    severity = severity.lower()
    if 'slight' in severity:
        return 'slight'
    elif 'serious' in severity:
        return 'serious'
    elif 'fatal' in severity:
        return 'fatal'
    else:
        return severity
    
    
def format_time(time):
    """
    Format time if in format '0731 (rather than 07:31)
    """
    if time[0] == "'":
        time = f'{time[1:3]}:{time[3:]}'
    
    return time

In [166]:
collision_files = [f for f in os.listdir('../data/raw-data/') if 'collision' in f]
collision_schema = params['collision_columns']

dfs = []
for file in collision_files:
    n = 0
    cols = ['Unnamed:']
    while len([c for c in cols if 'Unnamed:' in c]) > 0:
        df = pd.read_csv(f'../data/raw-data/{file}', skiprows=n)
        cols = df.columns
        n += 1
        
    df = format_column_names(df)
    df = extract_columns(df, collision_schema)
    
    df['collision_severity'] = df['collision_severity'].apply(format_severity)
    
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True, dayfirst=True)
    
    df['time'] = df['time'].apply(format_time)
    df['time'] = pd.to_datetime(df['time']).dt.time
    
    for col in ['borough', 'location']:
        df[col] = df[col].apply(lambda x: x.upper())
    
    dfs.append(df)
    
    
collisions = pd.concat(dfs)
del dfs

collisions.shape

(270046, 8)

In [167]:
collisions.head()

,collision_index,borough,easting,northing,location,collision_severity,date,time
0,1150030969,CITY OF LONDON,531850,181710,LONG LANE J/W WEST SMITHFIELD,slight,2015-10-15,07:31:00
1,1150030972,CITY OF LONDON,532810,180870,122 CANNON STREET J/W KING WILLIAM STREET,slight,2015-10-16,20:15:00
2,1150030975,CITY OF LONDON,533245,181510,BISHOPSGATE J/W LIVERPOOL STREET,serious,2015-10-20,15:26:00
3,1150031496,CITY OF LONDON,531780,181500,LONDON WALL 5 METRES WORMWOOD STREET,slight,2015-10-29,16:09:00
4,1150031510,CITY OF LONDON,533060,181437,OLD BROAD STREET 10 METRES SOUTH OF JUNCTION ...,slight,2015-10-30,08:50:00
